In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import translators as ts
import json
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

Using South Korea server backend.


In [2]:
url="https://ieeexplore.ieee.org/document/9411225"
url="https://ieeexplore.ieee.org/document/7809977" #会议
url="https://ieeexplore.ieee.org/document/5165285" #TWC
url='https://ieeexplore.ieee.org/document/4343975'
url='https://ieeexplore.ieee.org/document/4347633'
#url='https://ieeexplore.ieee.org/document/8524609'  #没有IEE参考文献的文章
#url='https://ieeexplore.ieee.org/document/10018916/' #该文章只有两列关键词 且没有参考文献
url=url+'references#references' if url[-1]=='/' else url+'/references#references'
html=urlopen(url)
html_byte=html.read()

service = ChromeService(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
driver = webdriver.Chrome(
    options=options,
    service=service
                          )
#driver = webdriver.Chrome(executable_path=".\\chromedriver.exe",options=webdriver.ChromeOptions().add_argument("--headless"))# type: ignore
driver.get(url)
wait = WebDriverWait(driver, 20) 
wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")

'''wait = WebDriverWait(driver, 20) # 设置最多等待10秒
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "stats-reference-link-viewArticle"))) # 等待id为some_id的元素出现
'''
html_driver = driver.page_source
driver.quit()

bs=BeautifulSoup(html_byte)
bs_driver=BeautifulSoup(html_driver,'lxml' )

In [3]:
f=open(".\\html\\get detail test.html_driver","wb")
f.write(html_byte)
f.close()

In [4]:
f=open(".\\html\\get detail test2.html","w")
f.write(html_driver)
f.close()


获取摘要和翻译

In [5]:
abstract = bs.find('meta', {'property': 'twitter:description'})['content']#.replace('n','') # type: ignore
print(abstract)

abstract_translation=ts.google(abstract,from_language="en",to_language='zh', host_url="https://translate.google.com/")
print(abstract_translation)

Multi-station passive localization algorithms based on hybrid Time Difference of Arrival (TDOA) and Angle of Arrival (AOA) have been thoroughly studied. But it usually requires each station to obtain the AOA of the source and relies on the precise station position. These conditions are generally not satisfied in practical applications. In addition, the geometry between the source and these stations also affects the localization accuracy. This paper studies the passive source localization problem based on multi-station TDOA and single-station AOA measurements. First, we propose a closed-form solution source localization method based on multi-station TDOA and single-station AOA measurements, which requires only the reference station used to calculate TDOA to be able to observe the AOA measurements of the source. It solves the problem of hybrid localization of different numbers of TDOA and AOA measurements. Theoretical analysis proves that the performance of the proposed method can reach 

获取关键词

In [13]:
match=re.search(r'"Author Keywords\s*","kwd":\[.*?\]',html_byte.decode(),re.DOTALL)
match=re.search(r'\[.*\]',match.group(),re.DOTALL)
if match:
    list=json.loads(match.group())
print(list)


['Localization', 'Time Difference of Arrival', 'Angle of Arrival', 'closed-form solution', 'Cramer-Rao Lower Bound', 'station position error', 'optimal geometry']


获取参考文献

In [26]:
View_Article=bs_driver.find_all("a",{"class":'stats-reference-link-viewArticle'})
CrossRef=bs_driver.find_all("a",{"class":'stats-reference-link-crossRef'})
View_Article_list=[]
CrossRef_list=[]

for article in View_Article:
    if article:
        View_Article_list.append(article.get("href"))
        
View_Article_list=['https://ieeexplore.ieee.org'+link for link in View_Article_list]

for article in CrossRef:
    if article:
        CrossRef_list.append(article.get("href"))

print(View_Article_list,CrossRef_list)

['https://ieeexplore.ieee.org/document/4449191', 'https://ieeexplore.ieee.org/document/1637927', 'https://ieeexplore.ieee.org/document/4610779'] ['https://doi.org/10.1121/1.1914426', 'https://doi.org/10.1007/978-3-662-11761-3']


获取期刊名

In [46]:
Journal_Title=bs_driver.find('strong', text='Published in: ').find_next_sibling().get_text()
print(Journal_Title)

'''impact_factor_list={'IEEE Transactions on Wireless Communications':10}
if Journal_Title in impact_factor_list:
    impact_factor=str(impact_factor_list[Journal_Title])
else:
    impact_factor='null'

print(impact_factor)'''

2016 Fourth International Conference on Ubiquitous Positioning, Indoor Navigation and Location Based Services (UPINLBS)


"impact_factor_list={'IEEE Transactions on Wireless Communications':10}\nif Journal_Title in impact_factor_list:\n    impact_factor=str(impact_factor_list[Journal_Title])\nelse:\n    impact_factor='null'\n\nprint(impact_factor)"

获取DOI

In [35]:
doi=bs_driver.find('strong', text='DOI: ').find_next_sibling().get_text()
Doi_link='https://doi.org/'+doi
print(Doi_link)

https://doi.org/10.1109/TWC.2009.070566


获取影响因子

In [30]:
print(bs_driver.find(href='/browse/conferences/title/')) #会议 
print(bs_driver.find({'href':'/browse/conferences/title/'}))   #TODO 为什么和上面输出不一样
print(bs_driver.find(href="/browse/periodicals/title/")) #期刊和杂志

#print(bs_driver.find(text='Journals & Magazines').parent) #这是不行的,他们都有这个,无法区分


None
None
<a _ngcontent-ijv-c160="" href="/browse/periodicals/title/">Journals &amp; Magazines</a>


In [31]:
Journal_index=False

if bs_driver.find(href="/browse/periodicals/title/"):
    
    Journal_index=True 
    print('this is a Journals or Magazines paper')

    journal_link='https://ieeexplore.ieee.org'+bs_driver.find('strong', text='Published in: ').find_next_sibling().get('href')
    print(journal_link)

    driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")
    driver.get(journal_link)
    time.sleep(5)
    journal_html = driver.page_source
    driver.quit()

    f=open(".\\html\\get detail test3.html","w")
    f.write(journal_html)
    f.close()

    bs_temp=BeautifulSoup(journal_html)
    impact_factor=bs_temp.find('a',{'class':'stats-jhp-impact-factor'}).find('span',{'class':'text-md-md-lh'}).get_text()
    
else :
    impact_factor=''

print(impact_factor)

this is a Journals or Magazines paper
https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=7693


C:\Users\Anable\AppData\Local\Temp\ipykernel_3992\1482078678.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=".\\chromedriver.exe")


8.346


获取日期

In [4]:

if not Journal_index:
    Date=str(bs_driver.find('strong', text='Date of Conference: ').next_sibling) # type: ignore
else:
    Date=str(bs_driver.find('strong', text='Date of Publication:').next_sibling) # type: ignore

print(Date)


 26-31 July 2007 


获取标题

In [33]:
print(bs.find('title').find_next_sibling().get('content')) # type: ignore #这里用.next_sibling会出错,因为两个元素间有空格

A new compact multichannel receiver for underwater wireless communication networks


获取文档PDF

In [4]:
doi='10.1109/TWC.2009.070566'
scihub_link='https://sci-hub.se/'+doi
pdf_html=urlopen(scihub_link).read()
bs_pdf=BeautifulSoup(pdf_html)

In [8]:
f=open('pdf.html','wb')
f.write(pdf_html)
f.close()

In [5]:
pdf_link='https:'+bs_pdf.find('embed',{'type':"application/pdf"}).get('src')
print(pdf_link)

https://zero.sci-hub.se/3452/c5f527d3fa8cc050a5e45aa2735b02ba/abdi2009.pdf#navpanes=0&view=FitH


In [7]:
import requests
import os

response = requests.get(pdf_link)

In [10]:
file_name = os.path.basename(pdf_link)+'.pdf'
with open('.\\pdf\\'+file_name, 'wb') as f:
    f.write(response.content)